In [ ]:
API_KEY = "YOUR_API_KEY"

In [ ]:
with open('4CDFIND01012020.txt', 'r') as fd:
    text = fd.read().replace('\n', ' ')

In [ ]:
!pip install pysbd

In [ ]:
from tika import parser

# Parse the PDF file and extract the text content
parsed_pdf = parser.from_file('4CDFIND01012020.pdf')
text = parsed_pdf['content']

In [ ]:
with open('tike-output.txt', 'w') as f:
    f.write(text)

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings

client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
                                    persist_directory="chroma",))

collection = client.create_collection("llm-hs-claims", embedding_function=embedding_functions.OpenAIEmbeddingFunction(
                    api_key=API_KEY,
                model_name="text-embedding-ada-002"
))

In [ ]:
from pysbd import Segmenter
import pprint

segmenter = Segmenter(language="en", clean=True)  # create a sentence segmenter for English
#text = "This is a sample text. It contains a few sentences. And it demonstrates the use of PySBD."
sentences = segmenter.segment(text)  # get a list of sentences

In [ ]:
chunk_size = 1000


for i in range(0, len(sentences), chunk_size):
    chunk = sentences[i:i+chunk_size]
    collection.add(
        documents = chunk,
        ids = [str(i) for i in range(i, i+len(chunk))],
        metadatas = [{"userID": "0", 'index' : str(i)} for i in range(len(chunk))]
    )

In [ ]:
TOP_K = 2

CONTEXT_WINDOW = 30


query_result = collection.query(
        query_texts=["mammogram"],
        n_results=TOP_K,
        where={"userID": "0"},
    )

#pprint.pprint(query_result)

for id in query_result['ids'][0]:
    index = int(id)
    pprint.pprint(' '.join(sentences[i] for i in range(index-CONTEXT_WINDOW//2, index+CONTEXT_WINDOW//2)))
    print()